In [1]:
## Update sklearn to prevent version mismatches:
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\rpmcl\anaconda3\envs\pythondata\lib\site-packages (0.0)


In [2]:
## Install joblib (to save model): 
## *Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

## Read CSV & Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("../Resources/Data/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
df.shape

(6991, 41)

## Set Target & Select Features

In [6]:
target = df['koi_disposition']
target.value_counts()

FALSE POSITIVE    3504
CONFIRMED         1800
CANDIDATE         1687
Name: koi_disposition, dtype: int64

In [10]:
features = df.drop(columns=['koi_disposition']).columns
len(features)

40

In [11]:
# Set features. This will also be used as your x values.
selected_features = df[features]

## Train / Test Split

In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features, target, random_state=42)

In [22]:
## Check split
print(round(len(X_train)/len(selected_features), 2))
print(round(len(X_test)/len(selected_features), 2))

0.75
0.25


In [23]:
print(round(len(y_train)/len(encoded_target), 2))
print(round(len(y_test)/len(encoded_target), 2))

0.75
0.25


## Pre-processing

In [55]:
## Encode Target
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

## Label-encode
label_encoder = LabelEncoder()
label_encoder.fit(target)
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

## One-hot encoding
y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

In [87]:
target_names = label_encoder.inverse_transform([0, 1, 2])
target_names

array(['CANDIDATE', 'CONFIRMED', 'FALSE POSITIVE'], dtype=object)

In [39]:
## Scale Features
from sklearn.preprocessing import MinMaxScaler

# Create MinMaxScaler model & Fit to training data
X_scaler = MinMaxScaler().fit(X_train)

# Transform training & testing data using X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Model Training

In [95]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

C:\Users\rpmcl\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(max_iter=1000)

In [96]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.6654587068472249
Testing Data Score: 0.6596109839816934


In [97]:
predictions = classifier.predict(X_test)
print(f"First 10 predictions: {predictions[:10].tolist()}")
print(f"First 10 real: {y_test[:10].tolist()}")

First 10 predictions: ['CANDIDATE', 'CANDIDATE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'CONFIRMED', 'FALSE POSITIVE', 'CANDIDATE', 'FALSE POSITIVE']
First 10 real: ['FALSE POSITIVE', 'CANDIDATE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'CANDIDATE', 'CANDIDATE', 'CANDIDATE', 'FALSE POSITIVE']


In [98]:
## Classification Report
from sklearn.metrics import classification_report
# predictions = model.predict(X_test)
print(classification_report(y_test, predictions, target_names=target_names))

                precision    recall  f1-score   support

     CANDIDATE       0.44      0.23      0.31       411
     CONFIRMED       0.64      0.68      0.66       484
FALSE POSITIVE       0.72      0.85      0.78       853

      accuracy                           0.66      1748
     macro avg       0.60      0.59      0.58      1748
  weighted avg       0.63      0.66      0.63      1748



In [99]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_scaled, y_train_encoded)

LogisticRegression(max_iter=1000)

In [100]:
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train_encoded)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test_encoded)}")

Training Data Score: 0.8483692542437535
Testing Data Score: 0.8443935926773455


In [101]:
predictions = classifier.predict(X_test_scaled)
print(f"First 10 predictions: {label_encoder.inverse_transform(predictions[:10]).tolist()}")
print(f"First 10 real: {label_encoder.inverse_transform(y_test_encoded[:10]).tolist()}")

First 10 predictions: ['FALSE POSITIVE', 'CANDIDATE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'CONFIRMED', 'CANDIDATE', 'CANDIDATE', 'FALSE POSITIVE']
First 10 real: ['FALSE POSITIVE', 'CANDIDATE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'CANDIDATE', 'CANDIDATE', 'CANDIDATE', 'FALSE POSITIVE']


In [104]:
## Classification Report
from sklearn.metrics import classification_report
# predictions = model.predict(X_test_scaled)
print(classification_report(y_test_encoded, predictions, target_names=target_names))

                precision    recall  f1-score   support

     CANDIDATE       0.70      0.63      0.67       411
     CONFIRMED       0.72      0.75      0.73       484
FALSE POSITIVE       0.98      1.00      0.99       853

      accuracy                           0.84      1748
     macro avg       0.80      0.79      0.80      1748
  weighted avg       0.84      0.84      0.84      1748



In [75]:
classifier.feature_importances_

AttributeError: 'LogisticRegression' object has no attribute 'feature_importances_'

### Add Residual Plot

## Model Tuning (Hyperparameters)

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

## Save Model

In [105]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
# joblib.dump(your_model, filename)
joblib.dump(classifier, filename)

['your_name.sav']